In [1]:
from openvqe.vqe import VQE

In [2]:
molecule_symbol = 'H2'
# In qubit ADAPT-VQE normally we choose the generalized single and double excitations
type_of_generator = 'singlet_gsd'
transform = 'JW'
active = False
opts = {
        'n_max_grads': 1,
        'optimizer': 'BFGS',
        'tolerance': 10**(-9),
        'type_conver': 'norm',
        'threshold_needed': 1e-7,
        'max_external_iterations': 29
    }

In [3]:
VQE('qubit_adapt', molecule_symbol, type_of_generator, transform, active, opts).execute()

 --------------------------------------------------------------------------
Running in the non active case: 
molecule symbol: H2 
molecule basis: 6-31g 
type of generator: singlet_gsd 
transform: JW 
options: {'n_max_grads': 1, 'optimizer': 'BFGS', 'tolerance': 1e-09, 'type_conver': 'norm', 'threshold_needed': 1e-07, 'max_external_iterations': 29} 
 --------------------------------------------------------------------------
 --------------------------------------------------------------------------
                                                          
                      Generate Hamiltonians and Properties from :
                                                          
 --------------------------------------------------------------------------
                                                          
Number of electrons =  2
Number of qubits before active space selection =  8
Orbital energies =  [-0.59279976  0.23579258  0.77956294  1.39138474]
Nuclear repulsion =  0.70556961

/home/nathan/miniconda/envs/openvqe/lib/python3.11/site-packages/openvqe/common_files/molecule_factory_with_sparse.py:488: UserWarning: This guess_init_state function is deprecated.
  _, _, hf_init = get_cluster_ops_and_init_guess(
/home/nathan/miniconda/envs/openvqe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


reference_energy from the simulator: -1.1265450345356895
reference_energy from the analytical calculations: -1.1265450345356895
 --------------------------------------------------------------------------
                                                          
                      Start Qubit ADAPT-VQE algorithm:
                                                          
 --------------------------------------------------------------------------
                                                          
 ------------------------------------------------------
        The number of maximum gradients inserted in each iteration: 1
 ------------------------------------------------------


 --------------------------------------------------------------------------
                         Qubit ADAPT-VQE iteration:  0
 --------------------------------------------------------------------------


 ------------------------------------------------------
        Start the analytical gradient c